In [13]:
import yaml
import os
import sys
import albumentations as A

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from building_footprint_segmentation.segmentation import init_segmentation, read_trainer_config
from building_footprint_segmentation.helpers.callbacks import CallbackList, load_callback

config_path = r"/home/ippws/building-footprint-segmentation/resnet50rio.yaml"

### Load the config file for training

In [14]:
config = read_trainer_config(config_path)

### Initalize type of segmentation to run for building footprint

In [15]:
segmentation = init_segmentation(config["Segmentation"])

### Load essential segmentation objects for training

In [ ]:
augmenters_alfa = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2)
])
augmenters = A.to_dict(augmenters_alfa)

model = segmentation.load_model(name=config["Model"]["name"])
criterion = segmentation.load_criterion(**config["Criterion"])
loader = segmentation.load_loader(
    config["Loader"]["root"],
    config["Loader"]["image_normalizer"],
    config["Loader"]["label_normalizer"],
    augmenters_alfa,
    config["Loader"]["batch_size"],
)
metrics = segmentation.load_metrics(
    data_metrics=config["Metrics"]
)
#if error on albumentations occur change augmenters_alfa to augmenters
optimizer = segmentation.load_optimizer(model, name=config["Optimizer"]["name"], lr= 1e-4, weight_decay = 1e-5)

callbacks = CallbackList()
# Ouptut from all the callbacks caller will be stored at the path specified in log_dir
for caller in config["Callbacks"]["callers"]:
    callbacks.append(load_callback(config["Callbacks"]["log_dir"], caller))

/home/ippws/anaconda3/envs/torchtest/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ippws/anaconda3/envs/torchtest/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Received kwargs: {'dice_weights': 1.0}


### Initalize the Trainer object


In [17]:
# Add this to your notebook after loading the config and before training

# 1. Import the new visualization classes
from building_footprint_segmentation.trainer import EnhancedTrainingVisualizer
from building_footprint_segmentation.helpers.callbacks import VisualizationCallback, AUCCallback
from building_footprint_segmentation.trainer import Trainer

# 2. Initialize the visualizer
visualizer = EnhancedTrainingVisualizer(save_dir='/home/ippws/building-footprint-segmentation/logs/graphs')

# 3. Create the callbacks
viz_callback = VisualizationCallback(
    log_dir=config["Callbacks"]["log_dir"],
    visualizer=visualizer,
    plot_frequency=5,  # Plot every 5 epochs
    collect_auc_data=True
)

auc_callback = AUCCallback(
    log_dir=config["Callbacks"]["log_dir"],
    visualizer=visualizer,
    compute_frequency=10  # Compute AUC every 10 epochs
)

# 4. Add these callbacks to your existing callbacks
callbacks.append(viz_callback)
callbacks.append(auc_callback)

# 5. Now create the trainer with all callbacks (including the new ones)
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,  # This now includes visualization callbacks
    optimizer=optimizer,
    scheduler=None,
)

# 6. IMPORTANT: Update callback parameters to pass the loader
callbacks.update_params({'loader': loader})

"""
trainer = Trainer(
    model=model,
    criterion=criterion,
    loader=loader,
    metrics=metrics,
    callbacks=callbacks,
    optimizer=optimizer,
    scheduler=None,
)
"""

'\ntrainer = Trainer(\n    model=model,\n    criterion=criterion,\n    loader=loader,\n    metrics=metrics,\n    callbacks=callbacks,\n    optimizer=optimizer,\n    scheduler=None,\n)\n'

### Start training


In [18]:

# 7. Train as normal
trainer.train(start_epoch=0, end_epoch=1)



Epoch 0, lr 0.0001: 100%|██████████| 7664/7664 [05:27<00:00, 23.39it/s, loss=0.48257]


>>Py-OneLiner : Loss : train: 0.48257012039422986, validation: 0.4595575865772035
>>Py-OneLiner : Train Metric : accuracy : 0.85774, precision : 0.75176, f1 : 0.78375, recall : 0.83809, iou : 0.65027
>>Py-OneLiner : Valid Metric : accuracy : 0.89383, precision : 0.82717, f1 : 0.84602, recall : 0.86703, iou : 0.73375
>>Py-OneLiner : Training Complete
>>Py-OneLiner : Run Time : 0:Days 0:Hours 5:Minutes 52:Seconds
Training Complete - Generating Final Visualizations
Training curves saved to: /home/ippws/building-footprint-segmentation/logs/graphs/final_training_curves.png
Visualization complete!


### Resuming Training
